In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# transformer encoder model code

In [3]:

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

def ViT(cf):
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(input_shape)     ## (None, 256, 3072)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)   ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model


if __name__ == "__main__":
    config = {}
    config["num_layers"] = 12
    config["hidden_dim"] = 512
    config["mlp_dim"] = 3072
    config["num_heads"] = 12
    config["dropout_rate"] = 0.1
    config["num_patches"] = 700
    config["patch_size"] = 16
    config["num_channels"] = 3
    config["num_classes"] = 2



# hyper parameter tuning according to dataloaders

In [4]:

""" Hyperparameters """
hp = {}
hp["w"]=700
hp["l"]=256
hp["num_channels"] = 3
hp["patch_size"] = 16
hp["num_patches_w"] = hp["w"] // hp["patch_size"]
hp["num_patches_l"] = hp["l"] // hp["patch_size"]
hp["num_patches"] = hp["num_patches_w"] * hp["num_patches_l"]
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 12
hp["lr"] = 1e-4
hp["num_epochs"] = 10
hp["num_classes"] = 2
hp["class_names"] = ["healthy","unhealthy"]

hp["num_layers"] = 12
hp["hidden_dim"] = 512
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

model = ViT(hp)
model.summary()
print("Model object loaded")

I0000 00:00:1717917635.862233    1550 service.cc:145] XLA service 0x5571af394810 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717917635.862301    1550 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1717917635.862306    1550 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1717917635.862312    1550 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1717917635.862314    1550 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1717917635.862317    1550 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1717917635.862320    1550 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1717917635.862323    1550 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1717917635.862325    1550 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 688, 768)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 688, 512)  │    393,728 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 688, 512)  │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_token         │ (None, 1, 512)    │          0 │ add[0][0]         │
│ (ClassToken)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 689, 512)  │          0 │ class_token[0][0… │
│ (Concatenate)       │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 689, 512)  │      1,024 │ concatenate[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 689, 512)  │ 12,601,856 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 689, 512)  │          0 │ multi_head_atten… │
│                     │                   │            │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 689, 512)  │      1,024 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 689, 3072) │  1,575,936 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 689, 3072) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 689, 512)  │  1,573,376 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 689, 512)  │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 689, 512)  │          0 │ dropout_2[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 689, 512)  │      1,024 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 689, 512)  │ 12,601,856 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 689, 512)  │          0 │ multi_head_atten… │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 689, 512)  │      1,024 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 689, 3072) │  1,575,936 │ layer_normalizat

 Total params: 189,434,370 (722.63 MB)

 Trainable params: 189,434,370 (722.63 MB)

 Non-trainable params: 0 (0.00 B)

Model object loaded


# important library for transformer code to run

In [5]:
!pip install patchify


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Training script ( with 8 batch size and 50 epochs )

In [1]:
import os
import numpy as np
import tensorflow as tf
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
from patchify import patchify
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.png")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["l"], hp["w"]))  # Notice the swap in dimensions
    image = image / 255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    patches = patches.reshape((hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"]))
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2]
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

def tf_dataset(images, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Paths """
    dataset_path = "/kaggle/input/complete-data-pics/complete_pics_data"
    model_path = "/kaggle/working/"
    csv_path = os.path.join("files", "log.csv")

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    train_ds = tf_dataset(train_x, batch=hp["batch_size"])
    valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])
    
    """ TPU Strategy """
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.TPUStrategy(resolver)
        print("All devices: ", tf.config.list_logical_devices('TPU'))
    except ValueError:
        strategy = tf.distribute.MirroredStrategy()  # Use MirroredStrategy for GPU or multi-GPU
        print("Running on GPU")

    with strategy.scope():
        """ Model """
        model = ViT(hp)
        model.load_weights(model_path)
        model.compile(
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            optimizer=tf.keras.optimizers.Adam(hp["lr"]),
            metrics=["acc"]
        )

    callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

    model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
    )

D0609 07:28:36.433484683    2865 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0609 07:28:36.433506493    2865 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0609 07:28:36.433510014    2865 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0609 07:28:36.433512706    2865 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0609 07:28:36.433515193    2865 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (default:ON)
D0609 07:28:36.433517873    2865 config.cc:196]                        gRPC EXPERIMENT client_privacy                         ON  (default:ON)
D0609 07:28:36.433520243    2865 config.cc:196]                        gRPC EXPERIMENT dapper_request_wire_size               OFF (default:O

Train: 736 - Valid: 92 - Test: 92


NameError: name 'hp' is not defined

# saving and loading model weights

In [3]:
model.load_weights('/kaggle/input/modelfile/model (3).keras')

# loading important libraries

In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from patchify import patchify
import tensorflow as tf

# testing classification prediction on single custom image

In [7]:
def process_image(path):
    """ Reading images """
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["l"], hp["w"]))  # Notice the swap in dimensions
    image = image / 255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    patches = patches.reshape((hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"]))
    patches = patches.astype(np.float32)

    return patches

In [11]:
sample_image_path = "/kaggle/input/sha-respp/sha_resp/healthy/102_1b1_Ar_sc_Meditron.png"  # Replace with the actual path to your sample image
patches = process_image(sample_image_path)

""" Make Prediction """
patches = np.expand_dims(patches, axis=0)  # Add batch dimension
prediction = model.predict(patches)

predicted_class_idx = np.argmax(prediction, axis=1)[0]
predicted_class = hp["class_names"][predicted_class_idx]

print(f"Predicted Class: {predicted_class}")
print(f"Prediction Scores: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Class: unhealthy
Prediction Scores: [[0.04093844 0.9590615 ]]
